In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [4]:
df = pd.read_csv("Boston.csv")
df.head()

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [5]:
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [6]:
df.shape

(506, 14)

In [7]:
df.isnull().sum()

crim       0
zn         0
indus      0
chas       0
nox        0
rm         0
age        0
dis        0
rad        0
tax        0
ptratio    0
black      0
lstat      0
medv       0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
crim       506 non-null float64
zn         506 non-null float64
indus      506 non-null float64
chas       506 non-null int64
nox        506 non-null float64
rm         506 non-null float64
age        506 non-null float64
dis        506 non-null float64
rad        506 non-null int64
tax        506 non-null int64
ptratio    506 non-null float64
black      506 non-null float64
lstat      506 non-null float64
medv       506 non-null float64
dtypes: float64(11), int64(3)
memory usage: 55.4 KB


In [24]:
y = df['medv']
X= df.drop('medv', axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .30, random_state=1)

In [25]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)
dtr

DecisionTreeRegressor()

In [26]:
print(dtr.score(X_test,y_test))

0.8333953276885044


In [27]:
from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10, None], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

In [28]:
rfr = RandomForestRegressor(random_state = 1)

In [38]:
#GridSearchCV
from sklearn.metrics import accuracy_score, r2_score
g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,cv = 3, n_jobs = 1, verbose = 0,return_train_score=True)
g_search.fit(X_train, y_train);
print(g_search.best_params_)
print(g_search.best_score_)
best = g_search.best_estimator_
print('R2: ', r2_score(y_pred = best.predict(X), y_true = y))

{'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'n_estimators': 13}
0.8225636920089764
R2:  0.9514624788219432


In [46]:
#RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 5 , stop = 15, num = 10)] # returns 10 numbers
max_features = ['auto', 'log2']
max_depth = [int(x) for x in np.linspace(5, 10, num = 2)]
max_depth.append(None)
bootstrap = [True, False]

r_grid = {'n_estimators': n_estimators,'max_features': max_features,'max_depth': max_depth,'bootstrap': bootstrap}
print(r_grid)

{'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15], 'max_features': ['auto', 'log2'], 'max_depth': [5, 10, None], 'bootstrap': [True, False]}


In [47]:
from sklearn.model_selection import RandomizedSearchCV
rfr_random = RandomizedSearchCV(estimator=rfr,param_distributions=r_grid, n_iter = 20,scoring='neg_mean_absolute_error', cv = 3, verbose=2, random_state=42,n_jobs=-1, return_train_score=True)
rfr_random.fit(X_train, y_train);
print(rfr_random.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


{'n_estimators': 12, 'max_features': 'auto', 'max_depth': None, 'bootstrap': True}


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.3s finished


In [48]:
print(rfr_random.score(X_test , y_test))

-2.3684210526315783
